In [1]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import re
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
%matplotlib inline

In [2]:
# Загрузка df для обучения
df_train = pd.read_csv('/data/share/lab04data/lab04_train.csv')
df_train.head(5)
df_train.shape

(320764, 117)

In [3]:
df_train = df_train[~df_train['TARGET'].isna()]
df_train['TARGET'].value_counts()

0.0    294607
1.0     26156
Name: TARGET, dtype: int64

In [ ]:
df_test = pd.read_csv('/data/share/lab04data/lab04_test.csv')
df_test.shape

In [ ]:
# смотрим данные
df_train.head(10).T

In [ ]:
df_train.info()

In [ ]:
df_train.describe(include=['int64'])

In [ ]:
df_train.describe(include=['object', 'bool'])

In [ ]:
#Check for na or missing data
def missing_values_table(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns

In [ ]:
missing_values_table(df_train)

In [ ]:
# Важно и нужно почистить данные
# Pipeline по очистке данных
# 1. Удалить пустые столбцы (если такие есть)
# 2. Remove the unnecessary column customerID - cleaned_df = df = df.drop('customerID', axis=1)
# 3. Convert all the non-numeric columns to numerical data types
# 4ю Преобразовать категориальные переменные

## Преобразование категориальных переменных - новое

In [4]:
cleaned_df = df_train

In [ ]:
# Преобразование категориальных переменных
# CLNT_TRUST_RELATION
# APP_MARITAL_STATUS
# APP_KIND_OF_PROP_HABITATION
# CLNT_JOB_POSITION_TYPE - Тип позиции
# CLNT_JOB_POSITION - Должность клиента
# APP_DRIVING_LICENSE - Признак наличия водительского удостоверения
# APP_EDUCATION - Образование клиента
# APP_TRAVEL_PASS - Признак "есть заграничный паспорт"
# APP_CAR - Признак наличия автомобиля
# APP_POSITION_TYPE - # CLNT_JOB_POSITION_TYPE??? похожее 'APP_POSITION_TYPE' его оставляю
# APP_EMP_TYPE - Тип работы (занятость)
# APP_COMP_TYPE - Тип работы (занятость) похожее
# PACK - Пакет услуг

In [5]:
cleaned_df[['APP_MARITAL_STATUS','CLNT_TRUST_RELATION', 'APP_KIND_OF_PROP_HABITATION',
          'CLNT_JOB_POSITION','APP_DRIVING_LICENSE',
          'APP_EDUCATION', 'APP_TRAVEL_PASS', 'APP_CAR',
          'APP_POSITION_TYPE', 'APP_EMP_TYPE', 'PACK']].head(10).T
# Отбросила: 'CLNT_JOB_POSITION_TYPE', 'APP_COMP_TYPE'
# Ниже для проверки по всем актегориальным переменным
# cleaned_df[['APP_MARITAL_STATUS','CLNT_TRUST_RELATION', 'APP_KIND_OF_PROP_HABITATION',
#           'CLNT_JOB_POSITION','APP_DRIVING_LICENSE',
#           'APP_EDUCATION', 'APP_TRAVEL_PASS', 'APP_CAR',
#           'APP_POSITION_TYPE','CLNT_JOB_POSITION_TYPE', 'APP_EMP_TYPE', 'APP_COMP_TYPE','PACK']].head(10).T

,0,1,2,3,4,5,6,7,8,9
APP_MARITAL_STATUS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,V,NaN
CLNT_TRUST_RELATION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FRIEND,NaN
APP_KIND_OF_PROP_HABITATION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RENT,NaN,NaN
CLNT_JOB_POSITION,NaN,офис-менеджер,консультант,NaN,экономист,NaN,Младший научный сотрудник,ДИРЕКТОР,СПЕЦИАЛИСТ,глав.бухгалтер
APP_DRIVING_LICENSE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN
APP_EDUCATION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UH,H,NaN
APP_TRAVEL_PASS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN
APP_CAR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN
APP_POSITION_TYPE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MANAGER,SPECIALIST,NaN
APP_EMP_TYPE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PRIVATE,PRIVATE,NaN


In [6]:
# Преобразования
changes={'':0}

RELATION = {'мать':'mother','друг':'friend','отец':'father', 'брат':'brother','близкий ро':'relative', 
            'сестра':'sister','дальний ро':'relative', 'сын':'son','дочь':'daughter','жена':'relative',
            'муж':'relative', 'мама':'mother'}

d = {'n' : 0, 'y' : 1}

In [7]:
cat_columns = ['APP_MARITAL_STATUS','CLNT_TRUST_RELATION', 'APP_KIND_OF_PROP_HABITATION',
          'CLNT_JOB_POSITION','APP_DRIVING_LICENSE',
          'APP_EDUCATION', 'APP_TRAVEL_PASS', 'APP_CAR',
          'APP_POSITION_TYPE', 'APP_EMP_TYPE', 'PACK']

!!!!!!!!!!!Уточнить насколько корректно cleaned_df[column].fillna(0, inplace = True) и потом использовать в Onehotencoding????

In [8]:
for column in cat_columns:#cleaned_df.columns:
    if cleaned_df[column].dtype == np.number:
        continue
#     cleaned_df[column] = LabelEncoder().fit_transform(cleaned_df[column])
    cleaned_df[column] = cleaned_df[column].str.lower()
    cleaned_df[column] = cleaned_df[column].str.strip()
    cleaned_df[column].fillna(0, inplace = True)
    cleaned_df = cleaned_df.replace({column: changes})

In [9]:
# Проверочка
cleaned_df[cat_columns].head(10).T

,0,1,2,3,4,5,6,7,8,9
APP_MARITAL_STATUS,0,0,0,0,0,0,0,m,v,0
CLNT_TRUST_RELATION,0,0,0,0,0,0,0,0,friend,0
APP_KIND_OF_PROP_HABITATION,0,0,0,0,0,0,0,rent,0,0
CLNT_JOB_POSITION,0,офис-менеджер,консультант,0,экономист,0,младший научный сотрудник,директор,специалист,глав.бухгалтер
APP_DRIVING_LICENSE,0,0,0,0,0,0,0,y,0,0
APP_EDUCATION,0,0,0,0,0,0,0,uh,h,0
APP_TRAVEL_PASS,0,0,0,0,0,0,0,n,0,0
APP_CAR,0,0,0,0,0,0,0,n,0,0
APP_POSITION_TYPE,0,0,0,0,0,0,0,manager,specialist,0
APP_EMP_TYPE,0,0,0,0,0,0,0,private,private,0


In [12]:
# missing_values_table(cleaned_df)

In [11]:
# Cтолбцы, где встречаются только 2 значения
cat_columns_2 = ['APP_CAR','APP_DRIVING_LICENSE','APP_TRAVEL_PASS']
# Cтолбцы, где встречаются только несколько значений
cat_columns_n = ['APP_MARITAL_STATUS','CLNT_TRUST_RELATION', 'APP_KIND_OF_PROP_HABITATION',
          'CLNT_JOB_POSITION','APP_EDUCATION', 'APP_POSITION_TYPE', 'APP_EMP_TYPE', 'PACK']

In [13]:
# Преобразование столбцов APP_CAR и APP_DRIVING_LICENSE
cleaned_df = cleaned_df.replace({'APP_CAR': d})
cleaned_df = cleaned_df.replace({'APP_DRIVING_LICENSE': d})
cleaned_df = cleaned_df.replace({'APP_TRAVEL_PASS': d})
cleaned_df['APP_DRIVING_LICENSE'].unique()

array([0, 1])

In [14]:
# из-за кривого написания заемщика делаю преобразование
cleaned_df = cleaned_df.replace({'CLNT_TRUST_RELATION': RELATION})

In [15]:
# делаю преобразование'CLNT_JOB_POSITION'
cleaned_df['CLNT_JOB_POSITION'] = cleaned_df['CLNT_JOB_POSITION'].astype(str).str.findall(r'[^0-9.,;_ \-/]+').apply(' '.join)

In [16]:
# работа со стоп-словам
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
stop_words = set(stopwords.words('russian'))
stop_words_new = set(["генеральный", "отдела", "глав", "младший", "продажам", "продаж", "научный", "офис", "главный",
                     "индивидуальный","коммерческий","ведущий","исполнительный", "финансовый", "фин", "технический", "заместитель",
                     "генерального", "системный"])
stop_words.update(stop_words_new)
def stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stop_words and len(word)>3])

In [17]:
cleaned_df['CLNT_JOB_POSITION'] = cleaned_df['CLNT_JOB_POSITION'].apply(stopwords)

In [18]:
top = list(cleaned_df['CLNT_JOB_POSITION'].value_counts()[cleaned_df['CLNT_JOB_POSITION'].value_counts()>100].index)

In [19]:
cleaned_df['TOP_JOB_POSITION'] = cleaned_df['CLNT_JOB_POSITION'].map( lambda x: x if x in top else 'other')

In [20]:
cleaned_df[['TOP_JOB_POSITION']].iloc[:5].values

array([[''],
       ['менеджер'],
       ['консультант'],
       [''],
       ['экономист']], dtype=object)

In [21]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [22]:
# LabelEncoder()
le_CLNT_TRUST_RELATION = LabelEncoder()
le_APP_MARITAL_STATUS = LabelEncoder()
le_APP_KIND_OF_PROP_HABITATION = LabelEncoder()
le_TOP_JOB_POSITION = LabelEncoder()
le_APP_EDUCATION = LabelEncoder()
le_APP_POSITION_TYPE = LabelEncoder()
le_APP_EMP_TYPE = LabelEncoder()
le_PACK = LabelEncoder()
cleaned_df['CLNT_TRUST_RELATION'] = le_CLNT_TRUST_RELATION.fit_transform(cleaned_df['CLNT_TRUST_RELATION'].astype(str))
cleaned_df['APP_MARITAL_STATUS'] = le_APP_MARITAL_STATUS.fit_transform(cleaned_df['APP_MARITAL_STATUS'].astype(str))
cleaned_df['APP_KIND_OF_PROP_HABITATION'] = le_APP_KIND_OF_PROP_HABITATION.fit_transform(cleaned_df['APP_KIND_OF_PROP_HABITATION'].astype(str))
cleaned_df['TOP_JOB_POSITION'] = le_TOP_JOB_POSITION.fit_transform(cleaned_df['TOP_JOB_POSITION'].astype(str))
cleaned_df['APP_EDUCATION'] = le_APP_EDUCATION.fit_transform(cleaned_df['APP_EDUCATION'].astype(str))
cleaned_df['APP_POSITION_TYPE'] = le_APP_POSITION_TYPE.fit_transform(cleaned_df['APP_POSITION_TYPE'].astype(str))
cleaned_df['APP_EMP_TYPE'] = le_APP_EMP_TYPE.fit_transform(cleaned_df['APP_EMP_TYPE'].astype(str))
cleaned_df['PACK'] = le_PACK.fit_transform(cleaned_df['PACK'].astype(str))

In [23]:
# Проверочка
cat_columns_new = ['APP_MARITAL_STATUS','CLNT_TRUST_RELATION', 'APP_KIND_OF_PROP_HABITATION',
          'TOP_JOB_POSITION','APP_DRIVING_LICENSE',
          'APP_EDUCATION', 'APP_TRAVEL_PASS', 'APP_CAR',
          'APP_POSITION_TYPE', 'APP_EMP_TYPE', 'PACK']
cat_columns_n_new = ['APP_MARITAL_STATUS','CLNT_TRUST_RELATION', 'APP_KIND_OF_PROP_HABITATION',
          'TOP_JOB_POSITION','APP_EDUCATION',
          'APP_POSITION_TYPE', 'APP_EMP_TYPE', 'PACK']
cleaned_df[cat_columns_new].head(10).T

,0,1,2,3,4,5,6,7,8,9
APP_MARITAL_STATUS,0,0,0,0,0,0,0,3,6,0
CLNT_TRUST_RELATION,0,0,0,0,0,0,0,0,4,0
APP_KIND_OF_PROP_HABITATION,0,0,0,0,0,0,0,4,0,0
TOP_JOB_POSITION,0,64,51,0,157,0,129,26,130,13
APP_DRIVING_LICENSE,0,0,0,0,0,0,0,1,0,0
APP_EDUCATION,0,0,0,0,0,0,0,11,5,0
APP_TRAVEL_PASS,0,0,0,0,0,0,0,0,0,0
APP_CAR,0,0,0,0,0,0,0,0,0,0
APP_POSITION_TYPE,0,0,0,0,0,0,0,1,3,0
APP_EMP_TYPE,0,0,0,0,0,0,0,3,3,0


In [24]:
ohe=OneHotEncoder()
temp_2 = ohe.fit_transform(cleaned_df[cat_columns_n_new]).toarray()
len(temp_2[0])

227

In [25]:
cleaned_df =  pd.concat([cleaned_df, pd.DataFrame(temp_2, index = cleaned_df.index)], axis=1)

## Работа с числовыми переменными

In [26]:
cleaned_df_new = cleaned_df.fillna(value=0)

In [27]:
cleaned_df_new.head(10).T

,0,1,2,3,4,5,6,7,8,9
Unnamed: 0,333149,303639,327113,384197,202462,53398,251372,71572,169965,257285
ID,479990,450480,473954,531038,349303,200239,398213,218413,316806,404126
CR_PROD_CNT_IL,0,0,0,0,0,0,0,1,3,0
AMOUNT_RUB_CLO_PRC,0,0,0,0,0,0,0.0185847,0.0280635,0,0.395599
PRC_ACCEPTS_A_EMAIL_LINK,0,0,0,0,0,0,0,0,0,0
APP_REGISTR_RGN_CODE,0,0,0,0,0,0,0,77,0,0
PRC_ACCEPTS_A_POS,0,0,0,0,0,0,0,0,0,0
PRC_ACCEPTS_A_TK,0,0,0,0,0,0,0,0,0,0
TURNOVER_DYNAMIC_IL_1M,0,0,0,0,0,0,0,0,0,0
CNT_TRAN_AUT_TENDENCY1M,0,0,0,0,0,0,0.5,0.142857,0,1


In [ ]:
cleaned_df_new.columns.values

In [ ]:
# # Поля для удаления
# # Числовые где больше нулей всего'CLNT_SALARY_VALUE','LDEAL_YQZ_COM','LDEAL_YQZ_CHRG','AVG_PCT_MONTH_TO_PCLOSE','MAX_PCLOSE_DATE',
# # 'AVG_PCT_DEBT_TO_DEAL_AMT','LDEAL_AMT_MONTH','LDEAL_YQZ_PC' 
# columns_to_delete = ['CLNT_SALARY_VALUE','LDEAL_YQZ_COM','LDEAL_YQZ_CHRG','AVG_PCT_MONTH_TO_PCLOSE','MAX_PCLOSE_DATE',
#                      'AVG_PCT_DEBT_TO_DEAL_AMT','LDEAL_AMT_MONTH','LDEAL_YQZ_PC'] 
# cleaned_df_new.drop(columns_to_delete, inplace=True, axis=1)
# cleaned_df_new.drop(cat_columns_n_new, inplace=True, axis=1)
# # + удаляю cat_columns_new так как их поменяла в onehotencoding

In [29]:
# Поля для удаления
# Числовые где больше нулей всего'CLNT_SALARY_VALUE','LDEAL_YQZ_COM','LDEAL_YQZ_CHRG','AVG_PCT_MONTH_TO_PCLOSE','MAX_PCLOSE_DATE',
# 'AVG_PCT_DEBT_TO_DEAL_AMT','LDEAL_AMT_MONTH','LDEAL_YQZ_PC' 
columns_to_delete = ['ID', 'APP_COMP_TYPE', 'CLNT_JOB_POSITION', 'CLNT_JOB_POSITION_TYPE', 'Unnamed: 0',
                    'CLNT_SALARY_VALUE','LDEAL_YQZ_COM','LDEAL_YQZ_CHRG','AVG_PCT_MONTH_TO_PCLOSE','MAX_PCLOSE_DATE',
                     'AVG_PCT_DEBT_TO_DEAL_AMT','LDEAL_AMT_MONTH','LDEAL_YQZ_PC'] 
cleaned_df_new.drop(columns_to_delete, inplace=True, axis=1)
cleaned_df_new.drop(cat_columns_n_new, inplace=True, axis=1)
# + удаляю cat_columns_new так как их поменяла в onehotencoding

In [ ]:
# cleaned_df_new.TARGET - в отдельную переменную, потом удалить

In [30]:
y = cleaned_df_new['TARGET']

In [31]:
cleaned_df_new.drop('TARGET', inplace=True, axis='columns')

In [32]:
from sklearn.model_selection import train_test_split

In [33]:
X_train, X_test, y_train, y_test = train_test_split(
    cleaned_df_new,
    y,
    test_size=0.5,
    random_state=0
)

In [ ]:
# from sklearn.ensemble import RandomForestClassifier as BlackBox
# model = BlackBox()
# # model = BlackBox(max_depth=5)
# model.fit(X_train, y_train)
# pred = model.predict(X_test)

In [ ]:
# pred

In [ ]:
# from sklearn.metrics import precision_score, \
#     recall_score, f1_score, accuracy_score

In [ ]:
# precision_score(y_test, pred), \
# recall_score(y_test, pred), \
# f1_score(y_test, pred), \
# accuracy_score(y_test, pred)

In [34]:
import xgboost as xgb
# xgb_model = xgb.XGBClassifier(max_depth=5, learning_rate=0.08, objective= 'binary:logistic',n_jobs=-1).fit(X_train, y_train)
model = xgb.XGBRegressor(n_estimators = 300, nthread= -1, max_depth = 3, objective = 'binary:logistic')
model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=300,
       n_jobs=1, nthread=-1, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [36]:
pred_xgb = model.predict(X_test)

In [37]:
from sklearn.metrics import roc_curve, roc_auc_score
roc_auc_score(y_test, pred_xgb)

0.8447477440786447